In [1]:
! pip install pandas numpy scikit-learn xgboost jieba nltk

In [2]:
# -*- coding: utf-8 -*-
import os
import re
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import jieba

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download("punkt_tab")

/home/b10173209/venv/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
[nltk_data] Downloading package punkt to /home/b10173209/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/b10173209/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/b10173209/nltk_data...
[nltk_data]   Package punkt_tab is alrea

True

In [3]:
# 定義主目錄
main_folder = '/home/b10173209/test.file/SS/class10(KFold)'

# 分類的清單
categories = ["anger","fear","joy","love","sadness","surprise"]

# 自定義 Tokenizer
def custom_tokenizer(text):
    tokens = jieba.lcut(text)
    return tokens

# 定義文本預處理函數
def preprocess_text(text):
    processed_text = text.lower()  # 將文本轉為小寫
    processed_text = re.sub(r'[^\w\s]', '', processed_text)  # 去除標點符號
    return processed_text

# 創建一個 DataFrame 來存儲結果
results_df = pd.DataFrame(columns=['Category', 'Split', 'Fold', 'Val Accuracy', 'Val Precision', 'Val Recall', 'Val F1 Score',
                                   'Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1 Score',
                                   'Val Macro Precision', 'Val Macro Recall', 'Val Macro F1',
                                   'Macro Precision', 'Macro Recall', 'Macro F1'])

# 定義參數網格
param_grid = {'learning_rate': [0.01, 0.05, 0.1],
              'max_depth': [3, 5, 7, 9],
              'n_estimators': [100, 200, 300]}

# 初始化 category_results 列表
category_results = []



In [4]:
# 迴圈跑 learning_rate、max_depth、n_estimators
for learning_rate in param_grid['learning_rate']:
    for max_depth in param_grid['max_depth']:
        for n_estimators in param_grid['n_estimators']:
            print(f'''
##################################################################################
##     Start Running learning_rate : {learning_rate}、max_depth : {max_depth}、n_estimators : {n_estimators}
################################################################################## '''
            )

            # 遍歷每個類別
            for category in categories:
                category_folder = os.path.join(main_folder, category)


                # 遍歷每個子資料夾
                for i in range(5):
                    subfolder = f'{category}_{i}'
                    subfolder_path = os.path.join(category_folder, subfolder)

                    # Debugging: Print folder path
                    print(f'資料夾路徑: {subfolder_path}')

                    # 檢查資料夾是否存在
                    if not os.path.exists(subfolder_path):
                        print(f'資料夾不存在: {subfolder_path}')
                        continue

                    # 讀取訓練集
                    train_file = f'train_{category}_{i}.csv'
                    train_df = pd.read_csv(os.path.join(subfolder_path, train_file))

                    # 讀取測試集
                    test_file = f'test_{category}_{i}.csv'
                    test_df = pd.read_csv(os.path.join(subfolder_path, test_file))

                    # 預處理文本數據
                    train_df['text'] = train_df['text'].apply(preprocess_text)
                    test_df['text'] = test_df['text'].apply(preprocess_text)

                    # 訓練集的標籤轉換為數字
                    train_labels = train_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()
                    test_labels = test_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                    # 使用 StratifiedKFold 進行分層交叉驗證
                    num_folds = 5
                    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

                    # 遍歷每一折
                    for fold, (train_index, val_index) in enumerate(skf.split(train_df, train_labels)):
                        print(f"折數: {fold + 1}/{num_folds}")

                        # 分割訓練和驗證集
                        train_data, val_data = train_df.iloc[train_index], train_df.iloc[val_index]

                        # 提取標籤
                        train_labels_fold = train_data[category].tolist()
                        
                        # 計算正類別與負類別的比例
                        positive_class_count = sum(train_labels_fold) 
                        negative_class_count = len(train_labels_fold) - positive_class_count 
                        
                        # 避免除以零的情況
                        if positive_class_count == 0:
                            scale_pos_weight = 0  
                        else:
                            scale_pos_weight = negative_class_count / positive_class_count  # 類別權重
        

                        # 訓練集的標籤轉換為數字
                        train_labels_fold = train_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                        # 驗證集的標籤轉換為數字
                        val_labels_fold = val_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                        # 建立 TfidfVectorizer 將文本數據轉換為 TF-IDF 特徵
                        vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=None)
                        X_train_fold = vectorizer.fit_transform(train_data['text'])
                        X_val_fold = vectorizer.transform(val_data['text'])
                        X_test = vectorizer.transform(test_df['text'])

                        xgb_model = xgb.XGBClassifier(learning_rate=learning_rate,
                                                      max_depth=max_depth,
                                                      n_estimators=n_estimators,
                                                      scale_pos_weight=scale_pos_weight,
                                                      random_state=42)

                        # 訓練 XGBoost 模型
                        xgb_model.fit(X_train_fold, train_labels_fold)

                        # 在驗證集上進行預測
                        predictions_val = xgb_model.predict(X_val_fold)

                        # 評估模型性能（在驗證集上）
                        accuracy_val = accuracy_score(val_labels_fold, predictions_val)
                        precision_val = precision_score(val_labels_fold, predictions_val, average='weighted')
                        recall_val = recall_score(val_labels_fold, predictions_val, average='weighted')
                        f1_val = f1_score(val_labels_fold, predictions_val, average='weighted')
                        print(f'Validation Accuracy: {accuracy_val}, Precision: {precision_val}, Recall: {recall_val}, F1 Score: {f1_val}')

                        # 在測試集上進行預測
                        predictions_test = xgb_model.predict(X_test)

                        # 評估模型性能（在測試集上）
                        accuracy_test = accuracy_score(test_labels, predictions_test)
                        precision_test = precision_score(test_labels, predictions_test, average='weighted')
                        recall_test = recall_score(test_labels, predictions_test, average='weighted')
                        f1_test = f1_score(test_labels, predictions_test, average='weighted')

                        # 計算 Validation Set 上的 Macro Precision、Macro Recall 和 Macro F1
                        macro_precision_val = precision_score(val_labels_fold, predictions_val, average='macro')
                        macro_recall_val = recall_score(val_labels_fold, predictions_val, average='macro')
                        macro_f1_val = f1_score(val_labels_fold, predictions_val, average='macro')

                        # 計算整體（Macro）的指標
                        macro_precision_test = precision_score(test_labels, predictions_test, average='macro')
                        macro_recall_test = recall_score(test_labels, predictions_test, average='macro')
                        macro_f1_test = f1_score(test_labels, predictions_test, average='macro')

                        # 將結果添加到 category_results 列表中
                        category_results.append({
                            'Category': category,
                            'Split': f'{category}_{i}',
                            'Fold': fold + 1,
                            'Val Accuracy': accuracy_val,
                            'Val Precision': precision_val,
                            'Val Recall': recall_val,
                            'Val F1 Score': f1_val,
                            'Test Accuracy': accuracy_test,
                            'Test Precision': precision_test,
                            'Test Recall': recall_test,
                            'Test F1 Score': f1_test,
                            'Val Macro Precision': macro_precision_val,
                            'Val Macro Recall': macro_recall_val,
                            'Val Macro F1': macro_f1_val,
                            'Macro Precision': macro_precision_test,
                            'Macro Recall': macro_recall_test,
                            'Macro F1': macro_f1_test,
                            'learning_rate': learning_rate,  # 新增的超參數欄位
                            'max_depth': max_depth,
                            'n_estimators': n_estimators
                        })

            # 將 category_results 轉換為 DataFrame
            category_results_df = pd.DataFrame(category_results)
            
            # 使用 pd.concat 合併數據
            results_df = pd.concat([results_df, category_results_df], ignore_index=True)

            # 清空 category_results 列表，以便下一個參數組合可以正確地存儲結果
            category_results.clear()

# 將結果 DataFrame 存儲到 Google Drive 中
results_df.to_excel('/home/b10173209/test.file/X/total_results.xlsx', index=False, encoding='utf-8')

# 顯示最終結果
print(results_df)

/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache



##################################################################################
##     Start Running learning_rate : 0.01、max_depth : 3、n_estimators : 100
################################################################################## 
資料夾路徑: /home/b10173209/test.file/SS/class10(KFold)/anger/anger_0
折數: 1/5


Loading model cost 0.354 seconds.
Prefix dict has been built successfully.


Validation Accuracy: 0.929375, Precision: 0.9266279349707347, Recall: 0.929375, F1 Score: 0.9223644397064252
折數: 2/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Validation Accuracy: 0.9746875, Precision: 0.9790921192930654, Recall: 0.9746875, F1 Score: 0.9763650080060531
折數: 5/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Validation Accuracy: 0.9771875, Precision: 0.9815512986480033, Recall: 0.9771875, F1 Score: 0.9787686377626288


TypeError: to_excel() got an unexpected keyword argument 'encoding'

In [5]:
# 將結果 DataFrame 存儲到 Google Drive 中
results_df.to_excel('/home/b10173209/test.file/X/total_results.xlsx', index=False)

# 顯示最終結果
print(results_df)

      Category       Split Fold  Val Accuracy  Val Precision  Val Recall  \
0        anger     anger_0    1      0.929375       0.926628    0.929375   
1        anger     anger_0    2      0.922813       0.921110    0.922813   
2        anger     anger_0    3      0.917813       0.913552    0.917813   
3        anger     anger_0    4      0.932500       0.932294    0.932500   
4        anger     anger_0    5      0.927188       0.924052    0.927188   
...        ...         ...  ...           ...            ...         ...   
5395  surprise  surprise_4    1      0.980625       0.981880    0.980625   
5396  surprise  surprise_4    2      0.980000       0.982763    0.980000   
5397  surprise  surprise_4    3      0.980625       0.981499    0.980625   
5398  surprise  surprise_4    4      0.974688       0.979092    0.974688   
5399  surprise  surprise_4    5      0.977187       0.981551    0.977187   

      Val F1 Score  Test Accuracy  Test Precision  Test Recall  Test F1 Score  \
0     